In [ ]:
#| all_slow

# Getting started

> A basic tutorial over key features in `ReLax`


This tutorial aims at introducing basis about `ReLax`, and 
how to use `ReLax` to generate counterfactual explanations 
for `jax`-based models.

In specific, we will cover followings in this tutorial:

1. Loading dataset with `TabularDataModule`;
2. Training machine learning classifier;
3. Generating counterfactual explanations;
4. Benchmarking different recourse methods.


## Preparation

We assume that you have already installed `ReLax`. 
If not, follow steps in [this installation tutorials](install.ipynb), or
just enter `pip install jax-relax`.

We also want to import some libraries for this tutorial. 

In [ ]:
import jax

## Load Dataset with `TabularDataModule`

`TabularDataModule` is a python class which modularizes tabular dataset loading. 
`TabularDataModule` loads a `.csv` file from the directory by specifying the following attributes:

- `data_name` is the name of your data.
- `data_dir` should be the directory of your data.
- `continous_cols` for continuous/numeric values in the data.
- `discret_cols` for discret values in the data. It will be converted to one-hot encoding for training purpose.
- `imutable_cols` for imutable values that we do not wish to change.


In [ ]:
from relax.data import TabularDataModuleConfigs, TabularDataModule, load_data

For example, to load the [adult](https://archive.ics.uci.edu/ml/datasets/adult) dataset,
we can specify the `TabularDataModuleConfigs` as

In [ ]:
data_config = TabularDataModuleConfigs(
    # The name of this dataset is "adult"
    data_name="adult",
    # The data file is located in `../assets/data/s_adult.csv`.
    data_dir="../assets/data/s_adult.csv",
    # Contains 2 features with continuous variables
    continous_cols=["age","hours_per_week"],
    # Contains 6 features with categorical (discrete) variables
    discret_cols=["workclass","education","marital_status","occupation","race","gender"],
    # Contains 2 features that we do not wish to change
    imutable_cols=["race", "gender"]
)

We can then pass `data_configs` to the `TabularDataModule`.

In [ ]:
datamodule = TabularDataModule(data_config)

Alternatively, we can also specify this config via a *dictionary*.

In [ ]:
# This approach is equivalent to using `TabularDataModuleConfigs`
data_config_dict = {
    # The name of this dataset is "adult"
    "data_name": "adult",
    # The data file is located in `../assets/data/s_adult.csv`.
    "data_dir": "../assets/data/s_adult.csv",
    # Contains 2 features with continuous variables
    "continous_cols": ["age","hours_per_week"],
    # Contains 6 features with categorical (discrete) variables
    "discret_cols": ["workclass","education","marital_status","occupation","race","gender"],
    # Contains 2 features which we do not wish to change
    "imutable_cols": ["race","gender"]
}
datamodule = TabularDataModule(data_config_dict)

For datasets supported by `ReLax`, we can simply call `load_data`:

In [ ]:
# This is equivalent to specify configs for `TabularDataModule`
datamodule = load_data('adult')

For more usage of loading datasets in `ReLax`, check out [the data module documentation](../01_data.module.ipynb).

## Train the Classifier
For the purpose of exposing full functionality of the framework, 
we will train the model using the built-in functions in `ReLax`,
which uses [haiku](https://dm-haiku.readthedocs.io/en/latest/index.html)
for building neural network blocks.
However, the recourse algorithms in `ReLax` can generate explanations 
for **all** jax-based framework (e.g., flax, haiku, vanilla jax). 

:::{.callout-warning}
The recourse algorithms in `ReLax` currently only supports binary classification. 
The output of the classifier must be a probability score (bounded by [0, 1]).
Future support for multi-class classification is planned.
:::

Training a classifier using the built-in functions in `ReLax` is very simple. 
We will first specify the classifier. The classifier is called `PredictiveTrainingModule`, 
which specifies the model structure, and the optimization procedure 
(e.g., it specifies the loss function for optimizing the model).
Next, we use `train_model` to train the model on `TabularDataModule`.


In [ ]:
from relax.module import PredictiveTrainingModuleConfigs, PredictiveTrainingModule
from relax.trainer import TrainingConfigs, train_model

### Define the Model

Defining `PredictiveTrainingModule` is similar with defining `TabularDataModule`.
We first specify the configurator as `PredictiveTrainingModuleConfigs`,
and pass this configurator to `PredictiveTrainingModule`.


In [ ]:
model_config = PredictiveTrainingModuleConfigs(
    lr=0.01, # Learning rate
    sizes=[50, 10, 50], # The sizes of the hidden layers
    dropout_rate=0.3 # Dropout rate
)

# specify the predictive model
module = PredictiveTrainingModule(model_config)

:::{.callout-note}
The training step for each batch is specified in `PredictiveTrainingModule`. 
Essentially, it will compute the binary cross-entropy loss for each batch, 
and apply backpropagation (via [adam](https://optax.readthedocs.io/en/latest/api.html#adam))
to update parameters of the model.
:::


### Train the Model


To train `PredictiveTrainingModule` for the entire dataset (specified in `TabularDataModule`),
we can simply call `train_model`:

In [ ]:
trainer_config = TrainingConfigs(
    n_epochs=10, # Number of epochs
    batch_size=256, # Batch size
    monitor_metrics='val/val_loss', # The metric to monitor
    logger_name='pred' # The name of the logger
)

# train the model
params, opt_state = train_model(
    module, datamodule, trainer_config
)

Epoch 9: 100%|██████████| 96/96 [00:02<00:00, 47.31batch/s, train/train_loss_1=0.0614]


### Make Predictions

The forward pass is done via `PredictiveTrainingModule.forward`. 
We wrap the `pred_fn` as follows:

In [ ]:
def pred_fn(x, params, rng_key):
    return module.forward(params, rng_key, x, is_training=False)

## Generate Counterfactual Explanations

In [ ]:
from relax.methods import VanillaCF
from relax.evaluate import generate_cf_explanations

Setup the counterfactual method. Here we use `VanillaCF`.

In [ ]:
cf_configs = { 
    'n_steps': 1000, # Number of steps
    'lr': 0.001 # Learning rate
}

cf_exp = VanillaCF(cf_configs)

Generate counterfactual examples.

In [ ]:
cf_results = generate_cf_explanations(
    cf_exp, datamodule, pred_fn, pred_fn_args={
        'params': params, 'rng_key': jax.random.PRNGKey(0)
})

100%|██████████| 1000/1000 [00:08<00:00, 113.06it/s]


## Benchmark the Counterfactual Method

After we obtain the counterfactual results, we can use  `benchmark_cfs` to evaluate the accuracy, validity, and proximity of the counterfactual example.

In [ ]:
from relax.evaluate import benchmark_cfs

In [ ]:
benchmark_cfs([cf_results])

,,acc,validity,proximity
adult,VanillaCF,0.825697,0.603489,7.2920003
